`text2vec-transformers` is **only** available through Weaviate open-source. Here are options to select your desired model: 

1. [Pre-built transformers model containers](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#pre-built-images)

2. [Any model from Hugging Face Model Hub](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#option-2-use-any-publicly-available-hugging-face-model)

3. [Use any private or local PyTorch or Tensorflow transformer model](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#option-3-custom-build-with-a-private-or-local-model)

## Dependencies

In [ ]:
!pip install --pre -I "weaviate-client==4.3.b2"

## Connect to Weaviate

In [ ]:
import weaviate

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local()

client.is_ready()

## Create a collection
> Collection stores your data and vector embeddings.

In [ ]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.
import weaviate.classes.config as wc

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_transformers( # specify the vectorizer and model type you're using
        pooling_strategy="masked_mean"
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

## Import the Data

In [ ]:
import requests, json
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

## Hybrid Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

### Hybrid Search only

The below query is finding Jeopardy questions about animals and is limiting the output to only two results. Notice `alpha` is set to `0.80`, which means it is weighing the vector search results more than bm25. If you were to set `alpha = 0.25`, you would get different results. 

In [ ]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search on a specific property

The `properties` parameter allows you to list the properties that you want bm25 to search on.

In [ ]:
response = jeopardy.query.hybrid(
    query="northern beast",
    query_properties=["question"],
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search with a `where` filter

Find Jeopardy questions about elephants, where the category is set to Animals.

In [ ]:
import weaviate.classes.query as wq # wq is an alias to save us from typing weaviate.classes everywhere ;)

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    filters=wq.Filter.by_property("category").equal("Animals"),
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search with a custom vector

You can pass in your own vector as input into the hybrid query, by using the `vector` parameter. 

In [ ]:
vector = [-0.119344814121723175, -0.22601557783782482, 0.343307773768901825, 0.421447690203785896, 0.51850304566323757, -0.04135638102889061, -0.004956562537699938, -0.0035841146018356085, 0.01667146012187004, -0.04788618162274361, -0.017431387677788734, 0.007470799144357443, -0.03725074976682663, -0.019192898645997047, 0.000902798434253782, -0.03436042740941048, -0.06718651205301285, -0.019682155922055244, 0.048777274787425995, 0.05929212644696236, -0.029497556388378143, -0.014709711074829102, 0.026938753202557564, 0.007368395570665598, -0.0031619099900126457, -0.045387931168079376, -0.02820613794028759, -0.0005118648405186832, -0.015748437494039536, 0.015419998206198215, -8.656250429339707e-05, -0.015787426382303238, 0.01223651971668005, 0.018071135506033897, -0.0389038622379303, -0.015234656631946564, 0.046179790049791336, -0.036981385201215744, 0.007422419264912605, 0.03681252524256706, 0.03271641209721565, -0.006550075951963663, 0.062385354191064835, -4.749358777189627e-05, -0.05315522477030754, -0.008986718952655792, 0.023720018565654755, 0.04003140702843666, -0.09639822691679001, 0.06366916000843048, -0.008965738117694855, -0.030423685908317566, 0.028025593608617783, -0.045483097434043884, 0.017829827964305878, 0.05015012249350548, 0.020414330065250397, 0.02445528656244278, 0.03232163190841675, -0.0011902153491973877, -0.012083040550351143, -0.03619984909892082, 0.028763433918356895, 0.04622166231274605, -0.017120877280831337, 0.026928355917334557, 0.027957390993833542, -0.007130495738238096, 0.040774088352918625, -0.013047979213297367, -0.02965693175792694, 0.010211427696049213, 0.01030472107231617, -0.007486498914659023, -0.035970158874988556, -0.03488561138510704, -0.008292327634990215, 0.01997722126543522, -0.01089937798678875, 0.04008284583687782, -0.026841484010219574, 0.0717591643333435, 0.07455220073461533, 0.05375919118523598, -0.06091667711734772, -0.004119484685361385, -0.02291276678442955, -0.004545729141682386, 0.003724967362359166, 0.02794576995074749, -0.02836090512573719, 0.0004210052720736712, 0.02694212645292282, 0.0039009954780340195, -0.008588884025812149, 0.03821517899632454, -0.003509549191221595, -0.036782994866371155, 0.0316341258585453, 0.003227108158171177, 0.044105395674705505, 0.02406424470245838, -0.023924732580780983, 0.07671564817428589, 0.014110337942838669, -0.03782413527369499, -0.032446376979351044, -0.01277676597237587, 0.051922328770160675, 0.00982830673456192, 0.05573970451951027, 0.013989139348268509, -0.022640923038125038, 0.03238740190863609, 0.005902513395994902, -0.06988172233104706, 0.02320891246199608, 0.013716954737901688, -0.036450739949941635, -0.08226403594017029, 0.010006032884120941, 0.037403807044029236, -0.004264313727617264, -0.025317920371890068, 0.026522569358348846, -0.025563620030879974, 0.008141961880028248, -0.018643252551555634, -0.002994611393660307, -0.02519265189766884, 0.004016819875687361, 0.010412836447358131, 0.02358146756887436, -0.01895127259194851, -0.00032174697844311595, 0.007694925647228956, 0.03519771248102188, 0.023803506046533585, 0.02199515886604786, -0.04145815223455429, -0.018939856439828873, 0.01673242822289467, 0.061465367674827576, 0.00578025309368968, 0.025567688047885895, 0.007762345019727945, -0.007705045398324728, 0.05053388699889183, 0.005971308331936598, 0.003737514838576317, 0.0050422814674675465, 0.019596261903643608, 0.030531145632267, -0.0314917154610157, 0.03265194967389107, 0.021070992574095726, -0.013981964439153671, 0.0019788951613008976, -0.04654382914304733, -0.02822619490325451, -0.03134949505329132, -0.044560398906469345, -0.03321956843137741, -0.040429286658763885, 0.0014711612602695823, 0.015237494371831417, -0.04400886595249176, -0.002490855986252427, 0.01586252823472023, 0.04582035169005394, -0.007193511817604303, -0.003710029413923621, -0.008810984902083874, 0.015817925333976746, -0.006300241686403751, 0.007918118499219418, -0.049017492681741714, 0.005970612168312073, -0.0295807346701622, 0.01990344375371933, -0.026403315365314484, 0.0424937903881073, 0.03970310464501381, 0.06851550936698914, 0.025686798617243767, -0.030576126649975777, -0.012233042158186436, 0.01660863123834133, 0.006317767780274153, -0.011326796375215054, 0.06967075914144516, 0.021824145689606667, -0.052172016352415085, -0.016605213284492493, -0.057098641991615295, -0.04872509464621544, 0.004453978501260281, 0.03154168277978897, 0.0354318842291832, 0.006747045088559389, 0.02130468189716339, 0.028476586565375328, -0.021551422774791718, -0.0507451631128788, 0.06333943456411362, 0.0020996564999222755, -0.01069617085158825, -0.0411326102912426, -0.020756758749485016, 0.021342871710658073, 0.08411172777414322, 0.001152115291915834, 0.02255486510694027, 0.017135266214609146, 0.037340257316827774, 0.01168818399310112, -0.05041133984923363, 0.03309524431824684, -0.005462253000587225, -0.03377050161361694, -0.0014419241342693567, -0.025505850091576576, -0.0038552305195480585, 0.02999795973300934, 0.0023868491407483816, 0.02144198678433895, -0.0023344820365309715, -0.06643835455179214, -0.03359115868806839, 0.03620823100209236, -0.004568215925246477, -0.058112844824790955, 0.014682049863040447, -0.07580640912055969, 0.04625740274786949, -0.010891472920775414, -0.05341479554772377, -0.06001722440123558, 0.02224947139620781, 0.041027236729860306, -0.00012153414718341082, 0.0037805328611284494, 0.014236964285373688, 0.02028162032365799, 0.007170515600591898, -0.023104431107640266, 0.04195636883378029, -0.0610978938639164, 0.05746160447597504, 0.0014240167802199721, -0.039536211639642715, 0.0016373955877497792, 0.0003072884283028543, -0.001608756952919066, 0.023871812969446182, 0.0345306359231472, -0.007415196392685175, 0.013668322935700417, 0.0025427518412470818, -0.017607342451810837, -0.11517762392759323, 0.023206369951367378, -0.02744770422577858, 0.002647802699357271, -0.03399734944105148, 0.008943461813032627, 0.03700130432844162, -0.02358066290616989, 0.05110080912709236, -0.003829334629699588, 0.02824871428310871, -0.0479760505259037, 0.009721203707158566, -0.029884107410907745, -0.04879360273480415, 0.00016358218272216618, 0.08167282491922379, 0.00023880600929260254, 0.006400170270353556, 0.041655413806438446, 0.02079765312373638, -0.018001805990934372, -0.026923127472400665, 0.023497717455029488, 0.003889981424435973, -0.014852370135486126, 0.015012676827609539, -0.0205772016197443, 0.020490232855081558, 0.00871635228395462, -0.03506492078304291, 0.0022999821230769157, 0.0011359852505847812, -0.027015866711735725, 0.002941967686638236, -0.010294473730027676, -0.04543664678931236, 0.003829958150163293, -0.02619173750281334, 0.035771988332271576, 0.06698797643184662, 0.052764538675546646, 0.03296105936169624, 0.024925170466303825, 0.014455633237957954, -0.04909819737076759, -0.01635417714715004, 0.008020590990781784, -0.05527082085609436, -0.02032247930765152, 0.006910940632224083, 0.014296498149633408, 0.020577076822519302, -0.06390856206417084, -0.030298175290226936, 0.00042066763853654265, -0.017291124910116196, 0.03255384415388107, -0.020768962800502777, 0.034829143434762955, 0.034188833087682724, -0.012843303382396698, -0.012185536324977875, -0.017695315182209015, -0.00594513351097703, 0.03002450242638588, 0.06913883984088898, 0.011171577498316765, -0.017164058983325958, 0.021870240569114685, -0.024416159838438034, -0.07240460813045502, -0.017782587558031082, -0.06271010637283325, -0.04153488576412201, 0.04262916371226311, -0.004655493423342705, 0.034259993582963943, 0.015554224140942097, 0.06917090713977814, -0.03754999116063118, -0.02526165544986725, 0.046117931604385376, -0.03680048882961273, -0.057557396590709686, -0.011507214978337288, -0.010854716412723064, -0.012757036834955215, 0.016887016594409943, -0.00389565946534276, -0.04382919520139694, -0.009573771618306637, 0.006608106661587954, 0.019878743216395378, -0.027216367423534393, 0.029854848980903625, -0.01676253043115139, -0.03367077559232712, 0.022410832345485687, 0.012690979987382889, -0.0038001874927431345, -0.040149252861738205, -0.028443550691008568, -0.008533579297363758, 0.022558776661753654, 0.03406636044383049, -0.038567714393138885, -0.07282765954732895, 0.05425000935792923, -0.01108078844845295, 0.04401346668601036, 0.05211755260825157, 0.009615067392587662, -0.007946817204356194, 0.028062807396054268, -0.01486042607575655, -0.013810168020427227, -0.027746928855776787, -0.05876319482922554, 0.003582334378734231, -0.012356484308838844, 0.023817170411348343, 0.004085152875632048, -0.01164256315678358, 0.0017092578345909715, 0.0378689244389534, 0.009032143279910088, -0.00795263797044754, 0.011862413957715034, -0.038974277675151825, -0.006320999003946781, 0.026455266401171684, -0.0023481647949665785, 0.015106679871678352, -0.0020351330749690533, -0.02412515878677368, 0.03389811888337135, 0.018361080437898636, 0.06143656000494957, 0.011456936597824097, 0.0028232645709067583, -0.028216496109962463, -0.026642611250281334, 0.02069086953997612, 0.0202633123844862, -0.019799936562776566, 0.03198038041591644, 0.027757005766034126, 0.05056627094745636, -0.048830531537532806, 0.019443975761532784, 0.03499466925859451, -0.05814236029982567, -0.03606671467423439, 0.04587607830762863, 0.07642633467912674, -0.08669358491897583, -0.04110484570264816, 0.013001208193600178, -0.0263389740139246, 0.04157758504152298, -0.01688331738114357, 0.04563450068235397, -0.006857412867248058, 0.001004301360808313, 0.0025637943763285875, -0.005182724446058273, -0.02615286223590374, -0.05504852905869484, -0.046326905488967896, -0.0061035663820803165, 0.018326912075281143, 0.005289848428219557, -0.00779558764770627, -0.009834988042712212, 0.0028476605657488108, -0.015114009380340576, 0.05853445082902908, -0.060266003012657166, -0.0007989748846739531, 0.013319953344762325, -0.027117284014821053, -0.009685675613582134, 0.025883691385388374, -0.024172982200980186, -0.05414188280701637, -0.018893929198384285, -0.06332946568727493, 0.01001776847988367, -0.026790570467710495, -0.017804889008402824, 0.03820061683654785, 0.04104139655828476, 0.0488925464451313, -0.08254847675561905, 0.04462799057364464, 0.002828360768035054, 0.017833350226283073, 0.05928749591112137, 0.04923329874873161, 0.07790961116552353, -0.028795232996344566, 0.007290772628039122, -0.0002288156101712957, 0.0064812214113771915, -0.03879395127296448, 0.04499180242419243, 0.03390586003661156, -0.044637493789196014, -0.018509214743971825, -0.006581888534128666, -0.027737220749258995, -0.05056362599134445, 0.02837003581225872, -0.008667782880365849, -0.04882919415831566, -0.03864400088787079, -0.04622940346598625, -0.04743894934654236, -0.02911391481757164, 0.014710553921759129, 0.043334126472473145, 0.06009296700358391, -0.03542754799127579, 0.011325509287416935, -0.03410780429840088, 0.034261345863342285, 0.003132296958938241, 0.0013371927198022604, -0.0482148677110672, -0.03314220532774925, 0.04771309345960617, -0.029727164655923843, 0.03538915887475014, 0.0020824891980737448, 0.02811354212462902, -0.020043376833200455, 0.05247418209910393, -0.021409984678030014, 0.0208491962403059, 0.07592733949422836, -0.03089182823896408, -0.006700906436890364, -0.012764113955199718, 0.009119303897023201, 0.0017575003439560533, -0.000705792976077646, -0.04018969088792801, -0.0650068074464798, 0.01851664111018181, -0.04554721713066101, 0.025455202907323837, -0.056414127349853516, 0.0072513907216489315, -0.0003037838905584067, -0.014373188838362694, 0.025162454694509506, -0.016032740473747253, 0.037562910467386246, 0.00539019051939249, 0.0069781276397407055, 0.019083788618445396, -0.015257495455443859, 0.014820081181824207, 0.033434707671403885, -0.005100224167108536, -0.04473336413502693, -0.029231861233711243, -0.02466806210577488, 0.03721855580806732, 0.04765034466981888, 0.03272223845124245, -0.05951588600873947, -0.030867258086800575, 0.09435411542654037, 0.04333748668432236, -0.015980303287506104, 0.049588464200496674, -0.0587925910949707, 0.03546220809221268, 0.01936286874115467, 0.004861677065491676, -0.0003613658482208848, 0.0658121332526207, 0.005216387566179037, -0.00026721894391812384, -0.0017125115264207125, -0.03988155350089073, 0.01311077456921339, 0.012286372482776642, -0.015207475982606411, 0.039687298238277435, 0.0030411162879318, -0.014219198375940323, 0.03164289519190788, -0.000306797242956236, 0.08280447870492935, 0.017930157482624054, 0.06629519164562225, -0.052393682301044464, -0.0006166169769130647, 0.0029258925933390856, -0.025678526610136032, -0.009381091222167015, -0.035653144121170044, 0.03743823245167732, -0.020674286410212517, -0.04858706146478653, 0.00906728208065033, 0.035401102155447006, -0.017265010625123978, 0.016732050105929375, -0.01617428846657276, -0.02709124982357025, -0.03511304780840874, -0.11904212832450867, 0.057321902364492416, -0.006758446339517832, 0.017145445570349693, 0.006529859732836485, 0.004917641635984182, -0.033229779452085495, -0.0049211252480745316, -0.028138330206274986, -0.00248794537037611, 0.03014036826789379, -0.003978196997195482, -0.006406700238585472, -0.06287303566932678, 0.0017207403434440494, 0.01699638180434704, -0.027868207544088364, -0.03235115110874176, 0.03671358525753021, 0.020288370549678802, -0.03672106936573982, -0.048421286046504974, 0.020062116906046867, 0.04803113639354706, 0.025054648518562317, 0.02986495941877365, 0.07462234050035477, 0.02512730099260807, -0.01183132454752922, -0.019132044166326523, -0.0056653995998203754, -0.027274802327156067, -0.0629994198679924, 0.04789704829454422, 0.02437557466328144, 0.005065961740911007, 0.019031979143619537, 0.01427337434142828, -0.0016519328346475959, 0.00516505166888237, -0.006725757382810116, 0.030652301385998726, -0.019582774490118027, -0.008145217783749104, 0.0011213838588446379, -0.0012979551684111357, 0.04814823344349861, 0.019276516512036324, 0.004105487372726202, 0.012042182497680187, -0.014933359809219837, -0.014367292635142803, 0.03400740772485733, 0.029001940041780472, 0.003446999005973339, -0.025387423112988472, -0.04492742940783501, -0.013550602830946445, -0.052827075123786926, -0.009753669612109661, 0.06643382459878922, -0.05484972521662712, 0.00888859573751688, -0.011477675288915634, 0.06935741007328033, -0.04802864417433739, -0.016385268419981003, -0.0436273068189621, -0.015805499628186226, -0.00467876810580492, 0.15997213125228882, -0.0023584368173033, -0.031141331419348717, -0.05694573000073433, 0.0044862269423902035, -0.01229949202388525, -0.03022519312798977, -0.02472771145403385, 0.02349107712507248, 0.01706554926931858, -0.008354868739843369, 0.015722665935754776, -0.045706987380981445, 0.006770753767341375, 0.002714993664994836, -0.02091672271490097, -0.0014497251249849796, -0.06404653936624527, 0.028254171833395958, -0.07679042220115662, -0.057926028966903687, 0.007130201440304518, 0.022160885855555534, -0.03203123062849045, 0.0309454333037138, 0.025610540062189102, -0.018444474786520004, 0.004960146266967058, 0.0028906313236802816, -0.0055052838288247585, -0.001733753364533186, 0.050401076674461365, 0.028762128204107285, -0.013507693074643612, -0.04443960636854172, 0.018915550783276558, -0.0051679159514606, 0.007091799285262823, 0.012077897787094116, 0.006963720545172691, 0.02301912195980549, -0.026824964210391045, 0.02366878278553486, 0.10567225515842438, -0.018456216901540756, -0.006178895477205515, 0.04002848267555237, 0.03371391445398331, -0.0028315645176917315, 0.006719862576574087, -0.045943815261125565, -0.039806291460990906, 0.02209482342004776, -0.0015198223991319537, 0.04574437066912651, -0.0021911533549427986, 0.011691724881529808, 0.014278598129749298, 0.05458660423755646, -0.0031955407466739416, 0.014233957976102829, 0.05085885152220726, -0.003701568813994527, 0.010289435274899006, -0.016152752563357353, -0.022694267332553864, 0.06488967686891556, -0.003152723191305995, 0.04806722700595856, -0.03656838834285736, -0.0033963427413254976, -0.05689616873860359, 0.05601230263710022, 0.03422895446419716, -0.04016558825969696, -0.019189929589629173, 0.00183323014061898, -0.03189973160624504, 0.027736909687519073, -0.011781875975430012, -0.02109593339264393, -0.028977058827877045, -0.018563155084848404, 0.00998506136238575, 0.014516972936689854, 0.019803298637270927, -0.04113854467868805, -0.04168829321861267, -0.018911423161625862, 0.08439420908689499, 0.033915821462869644, 0.03135434910655022, 0.017903892323374748, -0.013166023418307304, -0.0381663478910923, -0.0345323346555233, 0.010339106433093548, -0.020886972546577454, 0.022095007821917534, 0.004448849707841873, 0.014483039267361164, -0.053995531052351, -0.042174797505140305, 0.0008551406790502369, -0.01555158942937851, -0.04966925084590912, -0.011437010020017624, 0.018865916877985, -0.027739156037569046, -0.03412230312824249, -0.010718933306634426, -0.004810512997210026, 0.01747693121433258, 0.021186230704188347, -0.033455990254879, 0.04161260649561882, -0.014990638010203838, -0.0010197567753493786, -0.062360383570194244, 0.007110854145139456, 0.004845837131142616, -0.058973997831344604, 0.029003040865063667, 0.035356659442186356, -0.008712791837751865, -0.02732003666460514, -0.02991781011223793, -0.04347563534975052, 0.033578816801309586, -0.001359485904686153, 0.01583167165517807, 0.02446107566356659, 0.01183009147644043, -0.005252493545413017, -0.05487408488988876, 0.024024296551942825, -0.011436565779149532, 0.01190025731921196, -0.026366932317614555, -0.007994825951755047, 0.008676636964082718, 0.0006670604343526065, 0.05505165830254555, 0.010281511582434177, -0.02940058708190918, -0.022170837968587875, -0.00018248996639158577, 0.010727016255259514, 0.001252843881957233, 0.006338000297546387, -0.03936925530433655, -0.009071432054042816, 0.016561763361096382, 0.0074938200414180756, 0.03811599686741829, -0.03745288401842117, 0.029484426602721214, -0.019096314907073975, -0.04971220716834068, 0.0373370498418808, -0.029737427830696106, 0.023224342614412308, 0.019529571756720543, 0.007332311011850834, 0.010644263587892056, -0.012313858605921268, 0.01842278055846691, -0.0015717479400336742, -0.004133040551096201, -0.014083676971495152, -0.0004601158434525132, 0.020756911486387253, -0.019226692616939545, 0.037885118275880814, 0.05481758713722229, 0.005579463671892881, 0.16166001558303833, 0.07388925552368164, 0.011564855463802814, -0.04628746584057808, -0.04627985507249832, -0.04640619456768036, -0.001722767949104309, 0.0010370882228016853, -0.020219650119543076, 0.02962341532111168, 0.028466446325182915, -0.026045307517051697, 0.028097430244088173, 0.006173413712531328, 0.03569236397743225, -0.011246714740991592, -0.0016757426783442497, -0.044785138219594955, -0.0010964175453409553, -0.04703965038061142, 0.0042718881741166115, -0.023152172565460205, 0.036958035081624985, -0.04285213723778725, 0.001560804434120655, 0.03792069852352142, 0.006896731909364462, 0.05043046548962593, -0.03475264459848404, -0.03129259869456291, -0.0031448404770344496, -0.03655010834336281, -0.0266928318887949, 0.01451115496456623, -0.020631417632102966, 0.007787240669131279, 0.03666854277253151, -0.04449746385216713, 0.02525128610432148, -0.0766909196972847, 0.006926719564944506, -0.02873622067272663, 0.02204776182770729, 0.022132771089673042, -0.02736222743988037, 0.037467241287231445, 0.01901089958846569, 0.040313027799129486, 0.012382233515381813, 0.013253538869321346, -0.05056605488061905, -0.041267916560173035, -0.04367375373840332, -0.004298580810427666, -0.0004336800775490701, -0.02234874665737152, -0.02150486223399639, -0.039257925003767014, -0.01554615143686533, -0.07693344354629517, -0.03136603906750679, -0.017251743003726006, -0.02265956811606884, 0.04061080887913704, 0.05788097903132439, -0.0537765808403492, -0.10143361240625381, -0.10424695163965225, 0.0405760258436203, 0.02934298664331436, -0.01745833456516266, 0.00978299230337143, -0.01356996688991785, 0.015220658853650093, 0.03480687737464905, -0.04472558572888374, -0.004704567138105631, 0.006168607622385025, 0.03226521238684654, 0.056532975286245346, -0.021481072530150414, -0.025087546557188034, 0.03289418667554855, 0.11109720915555954, 0.014381269924342632, 0.04701627790927887, 0.0011049493914470077, -0.011368435807526112, 0.043245580047369, -0.001733618788421154, -0.009559637866914272, 0.0038146066945046186, -0.00988641008734703, 0.004991834983229637, 0.019954632967710495, -0.014034979976713657, -0.020113134756684303, 0.002768103266134858, -0.025917336344718933, 0.0073610940016806126, -0.047405533492565155, 0.006559002213180065, 0.06497453153133392, 0.07453130930662155, 0.037656571716070175, 0.006289023905992508, -0.0069521511904895306, -0.025066541507840157, 0.023943033069372177, -0.03493541106581688, 0.007169375661760569, 0.02718634344637394, -0.019945301115512848, 0.049828510731458664, -0.023336904123425484, 0.003728412324562669, 0.022957749664783478, 0.048021212220191956, 0.03186311945319176, -0.03567328304052353, -0.043037720024585724, 0.03578183799982071, -0.059989843517541885, 0.03623444586992264, -0.008340297266840935, -0.024605009704828262, -0.045203883200883865, 0.04967484995722771, -0.035480957478284836, 0.016681257635354996, -0.03615349531173706, 0.10341845452785492, 0.016537152230739594, -0.028537368401885033, 1.4769671906833537e-05, 0.004542533773928881, -0.024518048390746117, 0.022093605250120163, 0.0037886376958340406, 0.022406937554478645, 0.0026841030921787024, -0.006061842665076256, 0.04349229857325554, -0.015352166257798672, -0.005391339305788279, -0.04211480915546417, 0.02179204672574997, -0.03341631591320038, 0.03257405012845993, -0.04897288978099823, 0.0034059640020132065, -0.04129614308476448, 0.0400514081120491, -0.00811777263879776, -0.008123376406729221, 0.031252194195985794, 0.0031671877950429916, 0.03144897520542145, 0.04636560380458832, 0.06220804154872894, 0.03305686265230179, -0.033477623015642166, -0.042896438390016556, -0.004487095400691032, -0.012724229134619236, 0.0704328641295433, -0.013197148218750954, -0.03402947634458542, 0.01143458392471075, 0.04720354825258255, -0.025294054299592972, -0.03024662472307682, -0.0016466985689476132, -0.024036487564444542, 0.011875392869114876, -0.048707589507102966, -0.030199982225894928, -0.007502291817218065, 0.05219845473766327, 0.038454268127679825, -0.002119684126228094, 0.01323720347136259, -0.038838259875774384, -0.027899669483304024, -0.011755617335438728, 0.008823170326650143, -0.01463047880679369, 0.02176562510430813, 0.032569557428359985, -0.012497995048761368, 0.03069465607404709, 0.007530884817242622, 0.017963960766792297, 0.017475232481956482, 0.03244347125291824, -0.021919693797826767, 0.004807836376130581, 0.043101780116558075, 0.05038371682167053, 0.0437183678150177, 0.052970852702856064, -0.00048519836855120957, -0.00218420522287488]

response = jeopardy.query.hybrid(
    query="animal",
    vector=vector,
    limit=2
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")